In [11]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime 
import os

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
print(tf.__version__)
print(tf.keras.__version__)

2.3.0
2.4.0


# temp

In [14]:
a = tf.constant([[1,2],[2,1]])
print(a)
print(tf.rank(a))

tf.Tensor(
[[1 2]
 [2 1]], shape=(2, 2), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


# 基础

## Tensor

In [55]:
x = tf.range(5)
x
net = layers.Embedding(10, 4)
net
out = net(x)
out
net.embeddings

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4], dtype=int32)>

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 0.02482803, -0.0047375 , -0.01536844, -0.04195168],
       [ 0.03208477, -0.03361894,  0.01295027,  0.00111902],
       [ 0.04154834, -0.03700677,  0.04317619, -0.0102105 ],
       [-0.04073603, -0.04494212, -0.00634596, -0.02684149],
       [ 0.00645452, -0.01360462, -0.01542989, -0.03752713]],
      dtype=float32)>

<tf.Variable 'embedding_6/embeddings:0' shape=(10, 4) dtype=float32, numpy=
array([[ 0.02482803, -0.0047375 , -0.01536844, -0.04195168],
       [ 0.03208477, -0.03361894,  0.01295027,  0.00111902],
       [ 0.04154834, -0.03700677,  0.04317619, -0.0102105 ],
       [-0.04073603, -0.04494212, -0.00634596, -0.02684149],
       [ 0.00645452, -0.01360462, -0.01542989, -0.03752713],
       [ 0.01036501,  0.00584296, -0.03298744,  0.04296329],
       [ 0.00091428,  0.02631411,  0.01078288, -0.01202569],
       [ 0.04824628, -0.03504296, -0.03065542,  0.0452172 ],
       [ 0.02297206,  0.03067997,  0.02673015, -0.0371575 ],
       [ 0.04667776,  0.02736947,  0.00525025, -0.01983349]],
      dtype=float32)>

### constant tensor

张量的数据类型和numpy.array基本一一对应

In [3]:
i = tf.constant(1) # tf.int32 类型常量
l = tf.constant(1,dtype = tf.int64) # tf.int64 类型常量
f = tf.constant(1.23) #tf.float32 类型常量
d = tf.constant(3.14,dtype = tf.double) # tf.double 类型常量
s = tf.constant("hello world") # tf.string类型常量
b = tf.constant(True) #tf.bool类型常量


print(tf.int64 == np.int64) 
print(tf.bool == np.bool)
print(tf.double == np.float64)
print(tf.string == np.unicode) # tf.string类型和np.unicode类型不等价

True
True
True
False


In [40]:
# input_shape = (2, 3, 4)
x1 = tf.reshape(tf.range(8),[2,4])
x2 = tf.reshape(tf.range(8),[2,4])
y = tf.keras.layers.Add()([x1, x2])
y

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[ 0,  2,  4,  6],
       [ 8, 10, 12, 14]], dtype=int32)>

In [28]:
def get_model():
    """函数式API搭建双塔DNN模型"""    
    # 输入
    user_id = keras.layers.Input(shape=(1,), name="user_id")
    gender = keras.layers.Input(shape=(1,), name="gender")
    
    # user 塔
    user_vector = tf.keras.layers.concatenate([
            layers.Embedding(1000, 100, input_length=10)(user_id), 
            layers.Embedding(1000, 2)(gender)
    ])
    
    return keras.models.Model(inputs=[user_id, gender], outputs=[user_vector]) 
model = get_model()
model.compile('rmsprop', 'mse')
input_array1 = np.random.randint(1000, size=(32, 1))
input_array2 = np.random.randint(1000, size=(32, 1))
output_array = model.predict([input_array1, input_array2])
print(output_array.shape)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 10, 100), (None, 1, 2)]

# RNN

In [58]:
net = keras.Sequential([ # 构建 2 层 RNN 网络

# 除最末层外，都需要返回所有时间戳的输出，用作下一层的输入

layers.SimpleRNN(64, return_sequences=True),

layers.SimpleRNN(1),

])

out = net(x) # 前向计算
out

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[-0.8778084 ],
       [ 0.6220725 ],
       [ 0.2937109 ],
       [ 0.31650898]], dtype=float32)>

### variable tensor

区别：
- 常量值不可以改变，常量的重新赋值相当于创造新的内存空间;变量的值可以改变，可以通过assign, assign_add等方法给变量重新赋值
- 变量能够被 TensorFlow 的自动求导机制所求导，常量不行

In [5]:
c = tf.constant([1.0,2.0])
print(c)
print(id(c))
c = c + tf.constant([1.0,1.0])
print(c)
print(id(c))
print('--------')
v = tf.Variable([1.0,2.0],name = "v")
print(v)
print(id(v))
v.assign_add([1.0,1.0])
print(v)
print(id(v))

tf.Tensor([1. 2.], shape=(2,), dtype=float32)
140338867020240
tf.Tensor([2. 3.], shape=(2,), dtype=float32)
140338867020432
--------
<tf.Variable 'v:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>
140338870593552


<tf.Variable 'UnreadVariable' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>

<tf.Variable 'v:0' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>
140338870593552


# how

## temp

## 构建数据dataset
Dataset数据结构应用非常灵活，因为它本质上是一个Sequece序列，其每个元素可以是各种类型，例如可以是张量，列表，字典，也可以是Dataset。

In [2]:
s1 = 'He likes cats'
s2 = 'He does not like cats'
s = s1.split(' ') + s2.split(' ')
s

['He', 'likes', 'cats', 'He', 'does', 'not', 'like', 'cats']

In [3]:
word_to_id = dict(zip(s, range(len(s))))
word_to_id

{'He': 3, 'likes': 1, 'cats': 7, 'does': 4, 'not': 5, 'like': 6}

## 线性回归（使用Sequential按层顺序构建模型）

In [ ]:
TRUE_W = 3.0
TRUE_b = 2.0
NUM_SAMPLES = 100

# 初始化随机数据
X = tf.random.normal(shape=[NUM_SAMPLES, 1]).numpy()
noise = tf.random.normal(shape=[NUM_SAMPLES, 1]).numpy()
y = X * TRUE_W + TRUE_b + noise  # 添加噪声

model = tf.keras.Sequential()
model.add(layers.Dense(32, input_dim = 1, activation='relu'))
# model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(1,))
model.summary()

model.compile(optimizer='sgd',
             loss='mse',
             metrics=['mse'])

modle
# model.fit(X, y, epochs=10, batch_size=100)
his = model.fit(X, y, epochs=10)
his.history

## 分类

### 情感分类-RNN（自定义model）

In [62]:
batchsz = 128 # 批量大小
total_words = 10000 # 词汇表大小 N_vocab
max_review_len = 80 # 句子最大长度 s，大于的句子部分将截断，小于的将填充
embedding_len = 100 # 词向量特征长度 n

# 加载 IMDB 数据集，此处的数据采用数字编码，一个数字代表一个单词
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=total_words)

# 打印输入的形状，标签的形状
print(x_train.shape, len(x_train[0]), y_train.shape)
print(x_test.shape, len(x_test[0]), y_test.shape)

# 截断和填充句子，使得等长，此处长句子保留句子后面的部分，短句子在前面填充
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)

# 构建数据集，打散，批量，并丢掉最后一个不够 batchsz 的 batch
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.shuffle(1000).batch(batchsz, drop_remainder=True)
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.batch(batchsz, drop_remainder=True)

(25000,) 218 (25000,)
(25000,) 68 (25000,)


In [70]:
class MyRNN(keras.Model):
    # Cell 方式构建多层网络
    def __init__(self, units):
        super().__init__()
        # [b, 64]，构建 Cell 初始化状态向量，重复使用
        self.state0 = [tf.zeros([batchsz, units])]
        self.state1 = [tf.zeros([batchsz, units])]
        # 词向量编码 [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        # 构建 2 个 Cell，使用 dropout 技术防止过拟合
        self.rnn_cell0 = layers.SimpleRNNCell(units, dropout=0.5)
        self.rnn_cell1 = layers.SimpleRNNCell(units, dropout=0.5)
        # 构建分类网络，用于将 CELL 的输出特征进行分类，2 分类
        # [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = layers.Dense(1)
    
    def call(self, inputs, training = None):
        x = inputs # [b, 80]
        # 获取词向量: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # 通过 2 个 RNN CELL,[b, 80, 100] => [b, 64]
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1): # word: [b, 100]
            out0, state0 = self.rnn_cell0(word, state0, training)
            out1, state1 = self.rnn_cell1(out0, state1, training)
        # 末层最后一个输出作为分类网络的输入: [b, 64] => [b, 1]
        x = self.outlayer(out1, training)
        # 通过激活函数，p(y is pos|x)
        prob = tf.sigmoid(x)
        
        return prob

units = 64 # RNN 状态向量长度 n
epochs = 20 # 训练 epochs
model = MyRNN(units) # 创建模型
# 装配
model.compile(optimizer = tf.optimizers.Adam(0.001), loss = tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

# 训练和验证
model.fit(db_train, epochs=epochs, validation_data=db_test)

# 测试
model.evaluate(db_test)

Epoch 1/20


TypeError: in user code:

    /Users/maowei/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-65-6b1c4b13d445>:28 call  *
        x = self.outlayer(out1, training)
    /Users/maowei/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:985 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)

    TypeError: call() takes 2 positional arguments but 3 were given


In [61]:
x_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

## layer

### 自定义layer

需要重写init、build、call方法。build方法主要是为了当你用到layer的时候，自适应input_shape。也可以写在init里，但是这样需要提前指定input_shape。很多时候我们会在model里调用layer，并不知道具体使用的时候input_shape是多少，所以建议还是写在build里

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    

x = tf.ones((2, 2))
# At instantiation, we don't know on what inputs this is going to get called
linear_layer = Linear(32)

# The layer's weights are created dynamically the first time the layer is called
y = linear_layer(x)


## 构建

In [2]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(16,)))
model.add(tf.keras.layers.Dense(32, activation='relu'))
# Now the model will take as input arrays of shape (None, 16)
# and output arrays of shape (None, 32).
# Note that after the first layer, you don't need to specify
# the size of the input anymore:
model.add(tf.keras.layers.Dense(32))

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                544       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
Total params: 1,600
Trainable params: 1,600
Non-trainable params: 0
_________________________________________________________________


In [4]:
model